In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

spark_home = os.environ.get('SPARK_HOME')

spark = SparkSession.builder \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()
loadConfigs(spark.sparkContext)

In [9]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [14]:
today = date.today().strftime('%Y%m%d')
# today = 20230602

input_folder = "gildings"
input_file = "gildings"
mode = "append"

df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/silver/{input_folder}/{input_file}_{today}")

In [15]:
df.show()

+-------+--------------------+------+------------------+-----------+---------+-------------+--------+
|post_id|              author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|  dateid|
+-------+--------------------+------+------------------+-----------+---------+-------------+--------+
|145qm88|     Homar-The-Whale|     0|{null, null, null}|       null|     null|         null|20230610|
|145qdyb|     FionaWalliceFan|     0|{null, null, null}|       null|     null|         null|20230610|
|145na3a|       sunnymushroom|     0|{null, null, null}|       null|     null|         null|20230610|
|145simo|        shodaimezack|     0|{null, null, null}|       null|     null|         null|20230610|
|145l0wp|  povitryana_tryvoga|     0|{null, null, null}|       null|     null|         null|20230610|
|145f0u1|             AFoxGuy|     0|{null, null, null}|       null|     null|         null|20230610|
|145g3bh|            chris_ro|     0|{null, null, null}|       null|     null|    

In [25]:
df_with_total_gildings = df.withColumn(
    'total_gildings',
    F.col('gildings.gid_1') + F.col('gildings.gid_2') + F.col('gildings.gid_3')
)

In [26]:
df_with_total_gildings.show()

+-------+--------------------+------+------------------+-----------+---------+-------------+--------+--------------+
|post_id|              author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|  dateid|total_gildings|
+-------+--------------------+------+------------------+-----------+---------+-------------+--------+--------------+
|145qm88|     Homar-The-Whale|     0|{null, null, null}|       null|     null|         null|20230610|          null|
|145qdyb|     FionaWalliceFan|     0|{null, null, null}|       null|     null|         null|20230610|          null|
|145na3a|       sunnymushroom|     0|{null, null, null}|       null|     null|         null|20230610|          null|
|145simo|        shodaimezack|     0|{null, null, null}|       null|     null|         null|20230610|          null|
|145l0wp|  povitryana_tryvoga|     0|{null, null, null}|       null|     null|         null|20230610|          null|
|145f0u1|             AFoxGuy|     0|{null, null, null}|       n

In [22]:
df.columns

['post_id',
 'author',
 'gilded',
 'gildings',
 'gild_silver',
 'gild_gold',
 'gild_platinum',
 'dateid']

In [24]:
from pyspark.sql import functions as F

df_gildings = df.groupBy('author').agg(
    F.sum('gildings.gid_1').alias('sum_gild_silver'),
    F.sum('gildings.gid_2').alias('sum_gild_gold'),
    F.sum('gildings.gid_3').alias('sum_gild_platinum')
).orderBy('author')